# Kernel Semântico com Integração do Servidor OpenBnB MCP

Este notebook demonstra como utilizar o Kernel Semântico com o servidor OpenBnB MCP real para pesquisar alojamentos reais do Airbnb usando o MCPStdioPlugin. Para acesso ao LLM, utiliza o Azure AI Foundry. Para configurar as suas variáveis de ambiente, pode seguir a [Lição de Configuração](/00-course-setup/README.md)


## Importar os Pacotes Necessários


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Criar a Conexão do Plugin MCP

Vamos conectar ao [servidor MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) utilizando o MCPStdioPlugin. Este servidor oferece funcionalidades de pesquisa do Airbnb através do pacote @openbnb/mcp-server-airbnb.


## Criar o Cliente

Neste exemplo, iremos utilizar o Azure AI Foundry para aceder ao nosso LLM. Certifique-se de que as suas variáveis de ambiente estão configuradas corretamente.


## Configuração do Ambiente

Configure as definições do Azure OpenAI. Certifique-se de que tem as seguintes variáveis de ambiente configuradas:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Compreender a Integração MCP do OpenBnB

Este notebook conecta-se ao **servidor real OpenBnB MCP**, que fornece funcionalidade de pesquisa do Airbnb.

### Como funciona:

1. **MCPStdioPlugin**: Utiliza comunicação via entrada/saída padrão com o servidor MCP
2. **Pacote NPM Real**: Faz o download e executa `@openbnb/mcp-server-airbnb` através do npx
3. **Dados em Tempo Real**: Retorna dados reais de propriedades do Airbnb a partir das suas APIs
4. **Descoberta de Funções**: O agente descobre automaticamente as funções disponíveis no servidor MCP

### Funções Disponíveis:

O servidor MCP do OpenBnB normalmente fornece:
- **search_listings** - Pesquisa propriedades do Airbnb por localização e critérios
- **get_listing_details** - Obtém informações detalhadas sobre propriedades específicas
- **check_availability** - Verifica disponibilidade para datas específicas
- **get_reviews** - Recupera avaliações de propriedades
- **get_host_info** - Obtém informações sobre os anfitriões das propriedades

### Pré-requisitos:

- **Node.js** instalado no seu sistema
- **Conexão à Internet** para fazer o download do pacote do servidor MCP
- **NPX** disponível (vem com o Node.js)

### Testar a Conexão:

Pode testar o servidor MCP manualmente executando:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Isto fará o download e iniciará o servidor MCP do OpenBnB, ao qual o Semantic Kernel se conecta para obter dados reais do Airbnb.


## Executar o Agente com o Servidor OpenBnB MCP

Agora vamos executar o Agente de IA que se conecta ao servidor OpenBnB MCP para procurar alojamentos reais no Airbnb em Estocolmo para 2 adultos e 1 criança. Sinta-se à vontade para alterar a lista `user_inputs` para modificar os critérios de pesquisa.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Resumo
Parabéns! Conseguiu criar um agente de IA que se integra com a pesquisa de alojamentos reais utilizando o Model Context Protocol (MCP):

## Tecnologias Utilizadas:
- Semantic Kernel - Para criar agentes inteligentes com Azure OpenAI
- Azure AI Foundry - Para capacidades de LLM e conclusão de chat
- MCP (Model Context Protocol) - Para integração padronizada de ferramentas
- OpenBnB MCP Server - Para funcionalidade real de pesquisa no Airbnb
- Node.js/NPX - Para executar o servidor MCP externo

## O que Aprendeu:
- Integração MCP: Conectar agentes do Semantic Kernel a servidores MCP externos
- Acesso a Dados em Tempo Real: Pesquisar propriedades reais do Airbnb através de APIs ao vivo
- Comunicação por Protocolo: Utilizar comunicação stdio entre o agente e o servidor MCP
- Descoberta de Funções: Descobrir automaticamente funções disponíveis nos servidores MCP
- Respostas em Streaming: Capturar e registar chamadas de funções em tempo real
- Renderização em HTML: Formatar respostas do agente com tabelas estilizadas e exibições interativas

## Próximos Passos:
- Integrar servidores MCP adicionais (meteorologia, voos, restaurantes)
- Construir um sistema multi-agente combinando protocolos MCP e A2A
- Criar servidores MCP personalizados para as suas próprias fontes de dados
- Implementar memória de conversação persistente entre sessões
- Desdobrar o agente em Azure Functions com orquestração de servidores MCP
- Adicionar autenticação de utilizadores e capacidades de reserva



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original no seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se uma tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes do uso desta tradução.
